In [1]:
install.packages("readxl")

Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
library("readxl")
df <- read_excel("../data/final_taaco.xlsx")

New names:
* `` -> ...1


## Computing Gist Inference Score (GIS)

In [3]:
# Referential Cohesion (PCREFz)
# Deep Cohesion (PCDCz)
# Verb Overlap LSA (SMCAUSlsa)
# Verb Overlap WordNet (SMCAUSwn)
# Word Concreteness (PCCNCz)
# Imageability for Content Words (WRDIMGc)
# Hypernymy for Nouns and Verbs (WRDHYPnv)

# computing z-scores
df["zSMCAUSlsa"] <- scale(df["SMCAUSlsa"], center = TRUE, scale = TRUE)
df["zSMCAUSwn"] <- scale(df["SMCAUSwn"], center = TRUE, scale = TRUE)
df["zWRDIMGc"] <- scale(df["WRDIMGc"], center = TRUE, scale = TRUE)
df["zWRDHYPnv"] <- scale(df["WRDHYPnv"], center = TRUE, scale = TRUE)

In [4]:
head(df,1)

...1,Unnamed: 0,url_post_id,raw_text,clean_text,DESPC,DESSC,DESWC,DESPL,DESPLd,...,pronoun_noun_ratio,repeated_content_lemmas,repeated_content_and_pronoun_lemmas,shared,share_subscriber,log_share_subscriber,zSMCAUSlsa,zSMCAUSwn,zWRDIMGc,zWRDHYPnv
0,0,02c5np6hzxtoosr_10114391|777658472415814,"Hey Gang, Our friend Steve Uesugi, shares this about Benzene, a toxin that creates Cancer is released from heated plastics in your car on a hot day. If your car is in the sun with the windows rolled up. You should roll down the windows and run the A/C for a few minutes to get rid of the Benzene and other toxic off gases released in your car on a hot day. Hugs Wes n Wanda Anzai Happy Healthy Hawaii The Institute of Advanced Anti-|Aging. DO YOU KNOW THE DANGER OF TURNING ON THE A/C AFTER STARTING THE ENGINE?. When we sit in our car and turn on the engine, we often, without thinking, turn on the A/C. Every time we do this, we put our health, and the health of the people around us, in danger. This occurs because.","Hey Gang, Our friend Steve Uesugi, shares this about Benzene, a toxin that creates Cancer is released from heated plastics in your car on a hot day. If your car is in the sun with the windows rolled up. You should roll down the windows and run the A/C for a few minutes to get rid of the Benzene and other toxic off gases released in your car on a hot day. Hugs Wes n Wanda Anzai Happy Healthy Hawaii The Institute of Advanced Anti-|Aging. DO YOU KNOW THE DANGER OF TURNING ON THE A/C AFTER STARTING THE ENGINE?. When we sit in our car and turn on the engine, we often, without thinking, turn on the A/C. Every time we do this, we put our health, and the health of the people around us, in danger. This occurs because.",3,7,144,2.333,1.528,...,0.09090909,0.2253521,0.2464789,1,0.00770416,-4.858261,-0.2888324,1.080884,1.044193,-0.4814615


In [5]:
for (i in 1:nrow(df)) {
    PCREFz <- df[i, "PCREFz"]
    PCDCz <- df[i, "PCDCz"]
    PCCNCz <- df[i, "PCCNCz"]
    zSMCAUSlsa <- df[i, "zSMCAUSlsa"]
    zSMCAUSwn <- df[i, "zSMCAUSwn"]
    zWRDIMGc <- df[i, "zWRDIMGc"]
    zWRDHYPnv <- df[i, "zWRDHYPnv"]
    gis <- PCREFz + PCDCz + (zSMCAUSlsa[1:1] - zSMCAUSwn[1:1]) - PCCNCz - zWRDIMGc[1:1] - zWRDHYPnv[1:1]
    df[i, "GIS"] <- gis
}

In [6]:
data <- df

# if only true and false
data <- df[which((df$rating==1 | df$rating==2)),]

# getting only shared urls
data <- df[which((df$shared>=1)),]

#data$shares[data$shares > 0] <- 1
y <- data["log_share_subscriber"]

# log of # of shares
# y <- log10(y)

# summary(glm(as.matrix(y) ~ as.matrix(data["GIS"]), family = "binomial"))
summary(m1 <- lm(as.matrix(y) ~ as.matrix(data["GIS"])))


Call:
lm(formula = as.matrix(y) ~ as.matrix(data["GIS"]))

Residuals:
    Min      1Q  Median      3Q     Max 
-7.7793 -1.1232  0.0841  1.2405  8.6796 

Coefficients:
                         Estimate Std. Error  t value Pr(>|t|)    
(Intercept)            -6.5482625  0.0302126 -216.740   <2e-16 ***
as.matrix(data["GIS"])  0.0007833  0.0088618    0.088     0.93    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.866 on 4010 degrees of freedom
  (6 observations deleted due to missingness)
Multiple R-squared:  1.948e-06,	Adjusted R-squared:  -0.0002474 
F-statistic: 0.007813 on 1 and 4010 DF,  p-value: 0.9296
